In [50]:
import sys
import os
import subprocess

from six import string_types


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import scipy
from skimage import io
from scipy import ndimage
from IPython.display import display
%matplotlib inline

In [51]:
from os import listdir

In [52]:
listdir('../input/planets-dataset/planet/planet/train-jpg')[:10]

In [53]:
len(listdir('../input/planets-dataset/planet/planet/train-jpg'))

# **Label Preparation**

In [54]:
labels_df = pd.read_csv("../input/planets-dataset/planet/planet/train_classes.csv")
labels_df.head()

In [55]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [56]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
# Display head
labels_df.head()

In [57]:
labels_df['image_name'] = labels_df['image_name'].apply(lambda x: '{}.jpg'.format(x)) 

In [58]:
y_col = list(labels_df.columns[2:])
y_col

In [59]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(labels_df, 
                                   test_size = 0.25, 
                                   random_state = 2018)
print('train', train_df.shape[0], 'validation', valid_df.shape[0])

In [60]:
train_df.head(3)

In [61]:
valid_df.head(3)

In [68]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = (128, 128)
datagen = ImageDataGenerator(
                                
                            rescale=1/255,
                              rotation_range=5,
                              shear_range=0.1,
                              zoom_range=0.1,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              horizontal_flip=True,
                              fill_mode='nearest')


train_gen = datagen.flow_from_dataframe(dataframe= train_df,
        directory='../input/planets-dataset/planet/planet/train-jpg/', 
                                        x_col='image_name', 
                                        y_col=y_col,
       target_size=IMG_SIZE, 
                                        class_mode='raw',
                                        batch_size= 32)
valid_datagen = ImageDataGenerator(rescale=1/255)

valid_gen = valid_datagen.flow_from_dataframe(dataframe= valid_df,
        directory='../input/planets-dataset/planet/planet/train-jpg/', 
                                        x_col='image_name', 
                                        y_col=y_col,
       target_size=IMG_SIZE,
                                        class_mode='raw',
                                         batch_size= 128)






In [69]:
import tensorflow as tf
from tensorflow.keras.layers import MaxPool2D, Conv2D, Dropout, BatchNormalization
resNet50V2_model=tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(128,128,3),
    pooling=None,
    classes=17,
    classifier_activation="softmax",
)
resNet50V2_model.trainable = True

In [64]:
resNet50V2_model.summary()

In [70]:
from keras import Sequential
from keras.layers import Dense,Flatten


model = Sequential()
model.add(Conv2D(512, kernel_size=(1,1), activation='relu'))
model.add(MaxPool2D((2,2), strides=2, padding='same'))
model.add(Dropout(0.8))
    
model.add(Conv2D(512, kernel_size=(1,1), activation='relu'))
model.add(MaxPool2D((2,2), strides=2, padding='same'))
model.add(Dropout(0.8))
    
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.8))
  
model.add(Flatten())
model.add(Dense(17, activation='softmax'))


learning_rate = 0.000001
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.CategoricalCrossentropy(), 
              metrics = ['accuracy'])
STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

In [71]:
history = model.fit(train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_gen ,
          validation_steps = STEP_SIZE_VALID,
          epochs = 1)

In [ ]:
history = history.history
n_epochs = len(history['loss'])

plt.figure(figsize=[14,4])
plt.subplot(1,2,1)
plt.plot(range(1, n_epochs+1), history['loss'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_loss'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(range(1, n_epochs+1), history['accuracy'], label='Training')
plt.plot(range(1, n_epochs+1), history['val_accuracy'], label='Validation')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
plt.legend()
plt.show()